<a href="https://colab.research.google.com/github/albertofalco/M7209/blob/main/M72_09_Actividad_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Gestión de datos no estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2023

Profesor: Facundo Santiago

Alumno: Alberto Falco
```

# Modelado Clásico - Actividad 1

Topic modeling es una técnica de aprendizaje automático no supervisado donde intentados descubrir tópicos que son abstractos al texto pero que pueden describir una colección de documentos. Es importante marcar que estos "tópicos" no son necesariamente equivalentes a la interpretación coloquial de tópicos, sino que responden a un patrón que emerge de las palabras que están en los documentos.

La suposición básica para Topic Modeling es que cada documento está representado por una mescla de tópicos, y cada tópico consite en una conlección de palabras.

## 1. Direcciones
Intentaremos construir un pipeline de machine learning donde como entrada recibamos texto, ejecutemos todos los pasos que vimos en este notebook incluyendo:

 - Eliminación de stopwords
 - Tokenización
 - Stemming y Lemmatization
 - Procesamiento especico del tema
 - Creación de features utilizando algun metodo de reducción de dimensionalidad, SVD, LSI, LDA

, para luego utilizar estas features para entrenar un modelo que nos permita predecir alguna propiedad interesante del set de datos. En este caso en particular, donde estermos analizando tweets, predeciremos el sector al que pertenece el tweet: Alimentación, Bebidas, etc.

En esta actividad les proponemos realizar cambios en alguna de las etapas del procesamiento para modificar la performance del modelo resultante y evaluar que cambios generan el mejor modelo resultante.

<img src='https://github.com/santiagxf/M72109/blob/master/docs/nlp/_images/classic_pipeline.png?raw=1' />

### 1.1. Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [ ]:
! wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv --quiet --no-clobber --directory-prefix ./Datasets/mascorpus/
! wget https://raw.githubusercontent.com/albertofalco/test/main/topic-modeling.txt --quiet --no-clobber

In [ ]:
! pip install -r topic-modeling.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 40.7 MB/s eta 0:00:00


In [ ]:
! python -m spacy download es_core_news_sm --quiet

2023-11-26 02:42:21.770934: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-26 02:42:21.771004: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-26 02:42:21.771062: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-26 02:42:21.784272: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-26 02:42:23.937407: W tensorflow/compiler/

In [ ]:
! wget https://raw.githubusercontent.com/santiagxf/M72109/master/m72109/nlp/normalization.py --quiet --no-clobber --directory-prefix ./m72109/nlp/

Primero importaremos algunas librerias necesarias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

### 1.2. Sobre el set de datos con el que vamos a trabajar

Utilizaremos como ejemplo un set de datos en español que contiene tweets que diferentes usuarios han publicado en relación a diferentes marcas de productos u empresas en el rubro de alimentación, construcción, automoviles, etc. Estos tweets, a su vez, están asociados a una de las diferentes fases en el proceso de ventas (también conocido como Marketing Funel) y por eso están tagueados con las fases de:
 - Awareness – el cliente es conciente de la existencia de un producto o servicio
 - Interest – activamente expresa el interes de un producto o servicio
 - Evaluation – aspira una marca o producto en particular
 - Purchase – toma el siguiente paso necesario para comprar el producto o servicio
 - Postpurchase - realización del proceso de compra. El cliente compara la diferencia entre lo que deseaba y lo que obtuvo

Referencia: [Spanish Corpus of Tweets for Marketing](http://ceur-ws.org/Vol-2111/paper1.pdf

> Nota: La version de este conjunto de datos que utilizaremos aqui es una versión preprocesada del original.

In [ ]:
tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

## 2. Desarrollo

### 2.1. Creando nuestros sets de datos de entrenamiento y testing

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'],
                                                    test_size=0.33,
                                                    stratify=tweets['SECTOR'])

### 2.2. Construcción del modelo: Pasos

**Paso 1:** Instanciamos nuestro preprocesamiento de texto

In [ ]:
from m72109.nlp.normalization import TweetTextNormalizer

normalizer = TweetTextNormalizer()

> Tip: Inspeccione todos los parametros de `TweetTextNormalizer`.

**Paso 2:** Instanciamos nuestro vectorizador, en este caso usando el método TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=True, sublinear_tf=True, norm='l2')

**Paso 3:** Instanciamos nuestro generador de features

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

featurizer = LatentDirichletAllocation()

**Paso 4:** Instanciamos nuestro clasificador que utilizará las features generadas hasta este momento

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

estimator = LogisticRegression()

### 2.3. Pipeline

Ensamblamos el pipeline

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[('normalizer', normalizer),
                          ('vectorizer', vectorizer),
                          ('featurizer', featurizer),
                          ('estimator', estimator)])

### 2.4. Evaluación

**Evaluación:** Entrenamos el modelo y testeamos su performance

In [ ]:
model = pipeline.fit(X=X_train, y=y_train)

100%|██████████| 2521/2521 [03:15<00:00, 12.90it/s]


In [ ]:
predictions = model.predict(X_test)

100%|██████████| 1242/1242 [01:29<00:00, 13.93it/s]


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

ALIMENTACION       0.00      0.00      0.00       110
  AUTOMOCION       0.41      0.09      0.15       148
       BANCA       0.33      0.10      0.15       198
     BEBIDAS       0.30      0.24      0.27       223
    DEPORTES       0.28      0.33      0.30       216
      RETAIL       0.25      0.66      0.36       268
       TELCO       0.00      0.00      0.00        79

    accuracy                           0.27      1242
   macro avg       0.22      0.20      0.18      1242
weighted avg       0.26      0.27      0.22      1242



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 3. Comparación automática de modelos

In [ ]:
# Importacion de librerias
from sklearn.model_selection import train_test_split
from m72109.nlp.normalization import TweetTextNormalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import itertools
import warnings

# Desactivacion de warnings.
warnings.filterwarnings("ignore")

# Split del dataset.
X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'],
                                                    test_size=0.33,
                                                    stratify=tweets['SECTOR'])

# Normalización.
normalizer = TweetTextNormalizer(language="spanish")
X_train_normalized = normalizer.fit_transform(X_train, y_train)
X_test_normalized = normalizer.fit_transform(X_test, y_test)

100%|██████████| 1242/1242 [01:13<00:00, 16.81it/s]


In [ ]:
# Diseño del espacio de busqueda.
search_space = {"vectorizer": [TfidfVectorizer(use_idf=True, sublinear_tf=True, norm='l2')],
                "featurizer": [LatentDirichletAllocation(n_jobs=-1),
                               LatentDirichletAllocation(n_jobs=-1, n_components=50),
                               LatentDirichletAllocation(n_jobs=-1, n_components=100),
                               LatentDirichletAllocation(n_jobs=-1, n_components=200),
                               TruncatedSVD(),
                               TruncatedSVD(n_components=50),
                               TruncatedSVD(n_components=100),
                               TruncatedSVD(n_components=200)
                               ],
                "estimator": [LogisticRegression(n_jobs=-1), GradientBoostingClassifier()]
                }
keys = list(search_space.keys())
values = list(search_space.values())
combinations = list(itertools.product(*values))
dict_list = [dict(zip(keys, combination)) for combination in combinations]

# Función para entrenamiento y predicción.
def train_predict(X_train, y_train, X_test, y_test, **kwargs):
  pipeline = Pipeline(steps=[('vectorizer', kwargs["vectorizer"]),
                             ('featurizer', kwargs["featurizer"]),
                             ('estimator', kwargs["estimator"])])
  model = pipeline.fit(X=X_train, y=y_train)
  predictions = model.predict(X_test)
  report = classification_report(y_test, predictions)
  return model, predictions, report

# Iteración sobre espacio de búsqueda.
results = []
for _, comb in enumerate(dict_list):
  result = {}
  result['Combination'] = comb
  print("Fitting model number {}: {}...".format(_, str(comb)))
  result['Info'] = train_predict(X_train_normalized, y_train, X_test_normalized, y_test, **comb)
  results.append(result)

Fitting model number 0: {'vectorizer': TfidfVectorizer(sublinear_tf=True), 'featurizer': LatentDirichletAllocation(n_jobs=-1), 'estimator': LogisticRegression(n_jobs=-1)}...
Fitting model number 1: {'vectorizer': TfidfVectorizer(sublinear_tf=True), 'featurizer': LatentDirichletAllocation(n_jobs=-1), 'estimator': GradientBoostingClassifier()}...
Fitting model number 2: {'vectorizer': TfidfVectorizer(sublinear_tf=True), 'featurizer': LatentDirichletAllocation(n_components=50, n_jobs=-1), 'estimator': LogisticRegression(n_jobs=-1)}...
Fitting model number 3: {'vectorizer': TfidfVectorizer(sublinear_tf=True), 'featurizer': LatentDirichletAllocation(n_components=50, n_jobs=-1), 'estimator': GradientBoostingClassifier()}...
Fitting model number 4: {'vectorizer': TfidfVectorizer(sublinear_tf=True), 'featurizer': LatentDirichletAllocation(n_components=100, n_jobs=-1), 'estimator': LogisticRegression(n_jobs=-1)}...
Fitting model number 5: {'vectorizer': TfidfVectorizer(sublinear_tf=True), 'feat

In [ ]:
# Obtención de resultados.
for result in results:
  print(result['Combination'], end = '\n')
  print(result['Info'][2])
  print('Number of components: {}'.format(result['Info'][0][1].n_components), end = '\n\n')

{'vectorizer': TfidfVectorizer(sublinear_tf=True), 'featurizer': LatentDirichletAllocation(n_jobs=-1), 'estimator': LogisticRegression(n_jobs=-1)}
              precision    recall  f1-score   support

ALIMENTACION       0.00      0.00      0.00       110
  AUTOMOCION       0.00      0.00      0.00       148
       BANCA       0.40      0.14      0.20       198
     BEBIDAS       0.31      0.21      0.25       223
    DEPORTES       0.31      0.37      0.34       216
      RETAIL       0.27      0.76      0.39       268
       TELCO       0.00      0.00      0.00        79

    accuracy                           0.29      1242
   macro avg       0.18      0.21      0.17      1242
weighted avg       0.23      0.29      0.22      1242

Number of components: 10

{'vectorizer': TfidfVectorizer(sublinear_tf=True), 'featurizer': LatentDirichletAllocation(n_jobs=-1), 'estimator': GradientBoostingClassifier()}
              precision    recall  f1-score   support

ALIMENTACION       0.15      

In [ ]:
# Almacenamiento de resultados.
import joblib

joblib.dump(results, 'results.joblib')

['results.joblib']

## 4. Responda las siguentes preguntas

> **Pista:** ¿Como podrían explorar esta multiplicidad de opciones de forma automática?

En particular deberan contestar las siguientes preguntas:

 - ¿Que métodos de reducción de dimensionalidad resultan mejores?

```
El método TruncatedSVD arrojó mejores resultados frente a Latent Dirichlet Allocation.
```

 - ¿Que numero de componentes hace sentido para estre problema?

```
De la ejecución del featurizer LDA, se obtuvo un número de componentes igual a 10. En cambio, para el
featurizer Truncated SVD, el número de componentes ideal es 2.

Sin embargo, se obtuvieron significativamente mejores resultados al utilizar un número mayor de componentes.
```

 - ¿Que tipo de modelos resultan mejores a la hora de ser utilizados como clasificadores? (estimator)

```
Ambas clases de estimadores arrojaron adecuados resultados cuando se utiliza un número de componentes lo
significativamente grande para mejorar las predicciones del modelo global de clasificación de tópicos.

El mejor pipeline obtenido comprendió el uso de TruncatedSVD como featurizer, con un número de componentes
igual a 200, y tomando como clasificador el modelo de regresión logística. El accuracy obtenido es de 0,94.
```

### Enviar trabajo práctico para evaluación

Para enviar sus respuestas:

1. Guarde una copia de este notebook en Google Colab.
2. Comparta el notebook y copie la dirección URL al miso.

  <img src='https://github.com/santiagxf/M72109/blob/master/docs/practice/_images/save_and_share.png?raw=1' width='700'/>

3. Genere una entrega en el campus pretando atención a:

  1. **Número de entrega** = 1.
  2. **Comentario** = pegue el link que acaba de copiar.

  <img src='https://github.com/santiagxf/M72109/blob/master/docs/practice/_images/share_and_submit.gif?raw=1' width='700'/>